Скрипт работает, но долго. Примерно 100 минут на Kaggle. Узким местом являются циклы.

df3['PosCashBalanceIDs'] = df3['PosCashBalanceIDs'].apply(lambda x: [eval(doc) for doc in x])

Подскажите пожалуйста, что с этой строкой кода не так и почему она не хочет работать с пользовательскими классами? 

In [1]:
import json
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder
# from category_encoders import TargetEncoder
import matplotlib.pyplot as plt
import seaborn as sns
import time
import json
import random
import warnings
warnings.filterwarnings('ignore')
from dataclasses import asdict
start_time = time.time()

In [2]:
uploaded= '/kaggle/input/shiftlogs/POS_CASH_balance_plus_bureau-001.log'

In [3]:
%%time
log_list = []
posh_log_list = []

with open(uploaded, 'r') as file:
    for line in file:
#         print(line)
        log_list.append(line)

# x = log_list[45052]                       # экономим время 1716501
# log_list = random.sample(log_list, 17165) # экономим время 1716501
# log_list.append(x)                        # экономим время 1716501

df = pd.DataFrame(log_list, columns=['log_list'])

CPU times: user 6.06 s, sys: 3.38 s, total: 9.43 s
Wall time: 39 s


In [4]:
%%time
df['log_list'] = df['log_list'].apply(lambda x: json.loads(x))

CPU times: user 119 ms, sys: 12.6 ms, total: 131 ms
Wall time: 150 ms


In [5]:
# делим датафрейм, вытаскиваем отдельно логи для POS_CASH_balance

df1 = df[df['log_list'].str['type']!='bureau']

df11 = df[df['log_list'].str['type']=='bureau']

In [6]:
%%time

df2 = pd.json_normalize(df1['log_list'])

df2

CPU times: user 929 µs, sys: 0 ns, total: 929 µs
Wall time: 912 µs


,type,data.CNT_INSTALMENT,data.records
0,POS_CASH_balance,31.0,"[{'CNT_INSTALMENT_FUTURE': 0.0, 'MONTHS_BALANC..."


In [7]:
def normalize_column(df: pd.DataFrame, column: str) -> pd.DataFrame:
    """Заменяет колонку со словарем на несколько колонок.
    
    Имена новых колонок - ключи словаря
    Значения в новых колонок - значения по заданному ключу в словаре
    """
    return pd.concat(
        [
            df, 
            pd.json_normalize(df[column]),
        ], 
        axis=1
    ).drop(columns=[column])

In [8]:
%%time

df3 = df2.explode('data.records')
df3.reset_index(drop=True)
df3 = normalize_column(df3.reset_index(drop=True), 'data.records')

df3

CPU times: user 34 ms, sys: 1.79 ms, total: 35.8 ms
Wall time: 47.5 ms


,type,data.CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,MONTHS_BALANCE,PosCashBalanceIDs,SK_DPD,SK_DPD_DEF
0,POS_CASH_balance,31.0,0.0,-10,"PosCashBalanceIDs(SK_ID_PREV=1974424, SK_ID_CU...",0,0
1,POS_CASH_balance,31.0,8.0,-9,"PosCashBalanceIDs(SK_ID_PREV=1921183, SK_ID_CU...",6,6
2,POS_CASH_balance,31.0,8.0,-16,"PosCashBalanceIDs(SK_ID_PREV=2713650, SK_ID_CU...",0,0
3,POS_CASH_balance,31.0,8.0,-8,"PosCashBalanceIDs(SK_ID_PREV=2817854, SK_ID_CU...",0,0
4,POS_CASH_balance,31.0,8.0,-10,"PosCashBalanceIDs(SK_ID_PREV=2610400, SK_ID_CU...",0,0
...,...,...,...,...,...,...,...
4249,POS_CASH_balance,31.0,12.0,-54,"PosCashBalanceIDs(SK_ID_PREV=1901396, SK_ID_CU...",0,0
4250,POS_CASH_balance,31.0,31.0,-76,"PosCashBalanceIDs(SK_ID_PREV=2831262, SK_ID_CU...",0,0
4251,POS_CASH_balance,31.0,31.0,-76,"PosCashBalanceIDs(SK_ID_PREV=2260348, SK_ID_CU...",0,0
4252,POS_CASH_balance,31.0,0.0,-4,"PosCashBalanceIDs(SK_ID_PREV=1236806, SK_ID_CU...",0,0


In [9]:
# 

from dataclasses import dataclass

@dataclass
class PosCashBalanceIDs:
    SK_ID_PREV: int
    SK_ID_CURR: int
    NAME_CONTRACT_STATUS: str
        
# df3['PosCashBalanceIDs'] = df3['PosCashBalanceIDs'].apply(lambda x: [eval(doc) for doc in x])

In [10]:
%%time

for n in range(len(df3)):
    df3['PosCashBalanceIDs'].loc[n] = eval(df3['PosCashBalanceIDs'].loc[n])
    
df3

CPU times: user 1.55 s, sys: 4.81 ms, total: 1.55 s
Wall time: 1.55 s


,type,data.CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,MONTHS_BALANCE,PosCashBalanceIDs,SK_DPD,SK_DPD_DEF
0,POS_CASH_balance,31.0,0.0,-10,"PosCashBalanceIDs(SK_ID_PREV=1974424, SK_ID_CU...",0,0
1,POS_CASH_balance,31.0,8.0,-9,"PosCashBalanceIDs(SK_ID_PREV=1921183, SK_ID_CU...",6,6
2,POS_CASH_balance,31.0,8.0,-16,"PosCashBalanceIDs(SK_ID_PREV=2713650, SK_ID_CU...",0,0
3,POS_CASH_balance,31.0,8.0,-8,"PosCashBalanceIDs(SK_ID_PREV=2817854, SK_ID_CU...",0,0
4,POS_CASH_balance,31.0,8.0,-10,"PosCashBalanceIDs(SK_ID_PREV=2610400, SK_ID_CU...",0,0
...,...,...,...,...,...,...,...
4249,POS_CASH_balance,31.0,12.0,-54,"PosCashBalanceIDs(SK_ID_PREV=1901396, SK_ID_CU...",0,0
4250,POS_CASH_balance,31.0,31.0,-76,"PosCashBalanceIDs(SK_ID_PREV=2831262, SK_ID_CU...",0,0
4251,POS_CASH_balance,31.0,31.0,-76,"PosCashBalanceIDs(SK_ID_PREV=2260348, SK_ID_CU...",0,0
4252,POS_CASH_balance,31.0,0.0,-4,"PosCashBalanceIDs(SK_ID_PREV=1236806, SK_ID_CU...",0,0


In [25]:
%%time

POS_CASH_balance = pd.DataFrame()

POS_CASH_balance["SK_ID_PREV"] = np.nan
for n in range(len(df3)):
    POS_CASH_balance['SK_ID_PREV'].loc[n] = df3['PosCashBalanceIDs'].loc[n].SK_ID_PREV
    
POS_CASH_balance["SK_ID_CURR"] = 0
for n in range(len(df3)):
    POS_CASH_balance['SK_ID_CURR'].loc[n] = df3['PosCashBalanceIDs'].loc[n].SK_ID_CURR
    
POS_CASH_balance['MONTHS_BALANCE'] = df3['MONTHS_BALANCE']
POS_CASH_balance['CNT_INSTALMENT'] = df3['data.CNT_INSTALMENT']

POS_CASH_balance['CNT_INSTALMENT_FUTURE'] = df3['CNT_INSTALMENT_FUTURE']

POS_CASH_balance["NAME_CONTRACT_STATUS"] = np.nan
for n in range(len(df3)):
    POS_CASH_balance['NAME_CONTRACT_STATUS'].loc[n] = df3['PosCashBalanceIDs'].loc[n].NAME_CONTRACT_STATUS

POS_CASH_balance['SK_DPD'] = df3['SK_DPD']
POS_CASH_balance['SK_DPD_DEF'] = df3['SK_DPD_DEF']

POS_CASH_balance

CPU times: user 5.11 s, sys: 5.54 ms, total: 5.11 s
Wall time: 5.12 s


,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,NaN,NaN,-10,31.0,0.0,Completed,0,0
1,NaN,NaN,-9,31.0,8.0,Active,6,6
2,NaN,NaN,-16,31.0,8.0,Active,0,0
3,NaN,NaN,-8,31.0,8.0,Active,0,0
4,NaN,NaN,-10,31.0,8.0,Active,0,0
...,...,...,...,...,...,...,...,...
4249,NaN,NaN,-54,31.0,12.0,Active,0,0
4250,NaN,NaN,-76,31.0,31.0,Active,0,0
4251,NaN,NaN,-76,31.0,31.0,Active,0,0
4252,NaN,NaN,-4,31.0,0.0,Completed,0,0


In [13]:
# 

from dataclasses import dataclass

@dataclass
class AmtCredit:
    CREDIT_CURRENCY: str
    AMT_CREDIT_MAX_OVERDUE: float
    AMT_CREDIT_SUM: float
    AMT_CREDIT_SUM_DEBT: float
    AMT_CREDIT_SUM_LIMIT: float
    AMT_CREDIT_SUM_OVERDUE: float
    AMT_ANNUITY: float
        
# df['AmtCredit'] = df['AmtCredit'].apply(lambda x: [eval(doc) for doc in x])

In [14]:
%%time

df11['AmtCredit'] = df11['log_list'].str['data'].str['record'].str['AmtCredit']       
for n in range(len(df11)):
    df11['AmtCredit'].loc[n] = eval(df11['AmtCredit'].loc[n])

CPU times: user 7.96 s, sys: 0 ns, total: 7.96 s
Wall time: 7.96 s


In [15]:
%%time

bureau = pd.DataFrame()

bureau['SK_ID_CURR'] = df11['log_list'].str['data'].str['record'].str['SK_ID_CURR']
bureau['SK_ID_BUREAU'] = df11['log_list'].str['data'].str['record'].str['SK_ID_BUREAU']
bureau['CREDIT_ACTIVE'] = df11['log_list'].str['data'].str['record'].str['CREDIT_ACTIVE']

bureau["CREDIT_CURRENCY"] = np.nan
for n in range(len(df11)):
    bureau['CREDIT_CURRENCY'].loc[n] = df11['AmtCredit'].loc[n].CREDIT_CURRENCY
    
bureau['DAYS_CREDIT'] = df11['log_list'].str['data'].str['record'].str['DAYS_CREDIT']
bureau['CREDIT_DAY_OVERDUE'] = df11['log_list'].str['data'].str['record'].str['CREDIT_DAY_OVERDUE']
bureau['DAYS_CREDIT_ENDDATE'] = df11['log_list'].str['data'].str['record'].str['DAYS_CREDIT_ENDDATE']
bureau['DAYS_ENDDATE_FACT'] = df11['log_list'].str['data'].str['record'].str['DAYS_ENDDATE_FACT']

bureau["AMT_CREDIT_MAX_OVERDUE"] = np.nan
for n in range(len(df11)):
    bureau['AMT_CREDIT_MAX_OVERDUE'].loc[n] = df11['AmtCredit'].loc[n].AMT_CREDIT_MAX_OVERDUE
    
bureau['CNT_CREDIT_PROLONG'] = df11['log_list'].str['data'].str['record'].str['CNT_CREDIT_PROLONG']

bureau["AMT_CREDIT_SUM"] = np.nan
for n in range(len(df11)):
    bureau['AMT_CREDIT_SUM'].loc[n] = df11['AmtCredit'].loc[n].AMT_CREDIT_SUM
    
bureau["AMT_CREDIT_SUM_DEBT"] = np.nan
for n in range(len(df11)):
    bureau['AMT_CREDIT_SUM_DEBT'].loc[n] = df11['AmtCredit'].loc[n].AMT_CREDIT_SUM_DEBT
    
bureau["AMT_CREDIT_SUM_LIMIT"] = np.nan
for n in range(len(df11)):
    bureau['AMT_CREDIT_SUM_LIMIT'].loc[n] = df11['AmtCredit'].loc[n].AMT_CREDIT_SUM_LIMIT
    
bureau["AMT_CREDIT_SUM_OVERDUE"] = np.nan
for n in range(len(df11)):
    bureau['AMT_CREDIT_SUM_OVERDUE'].loc[n] = df11['AmtCredit'].loc[n].AMT_CREDIT_SUM_OVERDUE
    
bureau['CREDIT_TYPE'] = df11['log_list'].str['data'].str['CREDIT_TYPE']
bureau['DAYS_CREDIT_UPDATE'] = df11['log_list'].str['data'].str['record'].str['DAYS_CREDIT_UPDATE']

bureau["AMT_ANNUITY"] = np.nan
for n in range(len(df11)):
    bureau['AMT_ANNUITY'].loc[n] = df11['AmtCredit'].loc[n].AMT_ANNUITY

bureau

CPU times: user 34.2 s, sys: 0 ns, total: 34.2 s
Wall time: 34.2 s


,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,352922,6192461,Active,currency 1,-904,0,NaN,NaN,NaN,0,112500.000,NaN,NaN,0.0,Credit card,-17,29172.465
1,238226,5059261,Closed,currency 1,-47,0,-17.0,-29.0,NaN,0,22500.000,0.0,0.0,0.0,Consumer credit,-9,NaN
2,223751,6376968,Closed,currency 1,-1132,0,9826.0,-958.0,0.0,0,0.000,NaN,NaN,0.0,Credit card,-958,NaN
3,392710,5786922,Closed,currency 1,-1460,0,-1094.0,-1093.0,NaN,0,63846.000,0.0,0.0,0.0,Consumer credit,-1093,NaN
4,196725,5059000,Closed,currency 1,-992,0,-625.0,-810.0,NaN,0,85968.000,0.0,0.0,0.0,Consumer credit,-748,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17160,386503,5299878,Closed,currency 1,-2067,0,-1337.0,-1618.0,NaN,0,202500.000,0.0,0.0,0.0,Consumer credit,-108,NaN
17161,410750,5092451,Closed,currency 1,-1514,0,-418.0,-850.0,NaN,0,445095.000,0.0,0.0,0.0,Consumer credit,-748,0.000
17162,448566,5959974,Closed,currency 1,-1872,0,-776.0,-1437.0,NaN,0,333000.000,0.0,0.0,0.0,Consumer credit,-498,13950.000
17163,399667,6707923,Closed,currency 1,-1396,0,-1030.0,-1242.0,NaN,0,191610.000,0.0,NaN,0.0,Consumer credit,-1236,0.000


In [16]:
POS_CASH_balance.to_csv('POS_CASH_balance.csv')
bureau.to_csv('bureau.csv')
end_time = time.time()
print("Execution time: ", (end_time - start_time)/60,"mins")

Execution time:  1.4712250630060832 mins
